In [ ]:
import numpy as np
import os
from utilities import *
import h5py

# Use os.path.join() to create the correct file path
filepath = get_filepath()
print(f"Base filepath: {filepath}")

# Path to the specific h5 file
h5_filepath = os.path.join(filepath, "Intra", "train", "rest_105923_1.h5")
print(f"H5 file path: {h5_filepath}")

# Load the data
data = read_h5py_file(h5_filepath)

# Check the data properties
print(f"Data min: {np.min(data)}, max: {np.max(data)}, mean: {np.mean(data)}")

# Z-score normalization (time-wise normalization as suggested in the document)
# This normalizes each sensor's time series independently
def z_score_normalize(data):
    """
    Perform z-score normalization on the data time-wise (for each sensor)
    
    Parameters:
    data (numpy.ndarray): Data with shape (n_sensors, n_timepoints)
    
    Returns:
    numpy.ndarray: Normalized data with the same shape
    """
    # Calculate mean and std for each sensor (row)
    mean = np.mean(data, axis=1, keepdims=True)
    std = np.std(data, axis=1, keepdims=True)
    
    # Replace zero std with 1 to avoid division by zero
    std[std == 0] = 1.0
    
    # Z-score normalization: (x - mean) / std
    normalized_data = (data - mean) / std
    
    return normalized_data

# Apply z-score normalization
normalized_data = z_score_normalize(data)

# Check the normalized data properties
print(f"Normalized data min: {np.min(normalized_data)}, max: {np.max(normalized_data)}, mean: {np.mean(normalized_data)}")

# Verify that the normalization worked as expected
# After z-score normalization, each row should have mean ≈ 0 and std ≈ 1
means = np.mean(normalized_data, axis=1)
stds = np.std(normalized_data, axis=1)
print(f"Mean of means: {np.mean(means)}, should be close to 0")
print(f"Mean of stds: {np.mean(stds)}, should be close to 1")



<class 'numpy.ndarray'>
(248, 35624)


In [ ]:
def load_and_normalize_files(directory_path, max_files=None, downsample_factor=None):
    """
    Load and normalize all h5 files in the specified directory
    
    Parameters:
    directory_path (str): Path to the directory containing h5 files
    max_files (int, optional): Maximum number of files to load
    downsample_factor (int, optional): Factor by which to downsample the data
    
    Returns:
    tuple: (data, labels) where data is a list of normalized matrices and labels are the corresponding task types
    """
    data_list = []
    labels = []
    
    # Get all h5 files in the directory
    h5_files = [f for f in os.listdir(directory_path) if f.endswith('.h5')]
    
    # Limit the number of files if specified
    if max_files is not None:
        h5_files = h5_files[:max_files]
    
    for file in h5_files:
        # Extract the task type from the filename
        task_type = file.split('_')[0]
        
        # Load the data
        file_path = os.path.join(directory_path, file)
        matrix = read_h5py_file(file_path)
        
        # Downsample if specified
        if downsample_factor is not None:
            matrix = matrix[:, ::downsample_factor]
        
        # Normalize the data
        normalized_matrix = z_score_normalize(matrix)
        
        # Add to lists
        data_list.append(normalized_matrix)
        labels.append(task_type)
    
    return data_list, labels

# Example usage for Intra-subject classification
intra_train_path = os.path.join(filepath, "Intra", "train")
intra_test_path = os.path.join(filepath, "Intra", "test")

# Load a small subset of files to test the function
# Using a downsample factor of 8 as suggested in section 3.1.2
train_data, train_labels = load_and_normalize_files(intra_train_path, max_files=3, downsample_factor=8)
test_data, test_labels = load_and_normalize_files(intra_test_path, max_files=2, downsample_factor=8)

# Print summary
print(f"\nLoaded {len(train_data)} training files and {len(test_data)} test files")
print(f"Training labels: {train_labels}")
print(f"Test labels: {test_labels}")
print(f"Shape of first training sample after downsampling: {train_data[0].shape}")